<font color="#CA3532"><h1 align="left">Deep Learning</h1></font>
<font color="#6E6E6E"><h2 align="left">Introducción a Keras: Autoencoders</h2></font>

Un autoencoder es una arquitectura de red neuronal utilizada para eliminar el ruido y la información irrelevante de los datos de entrada. La red está diseñada para aprender a reconstruir los datos de entrada a través de una representación comprimida (o codificación) en una capa intermedia. Este modelo se entrena para generar, a partir de los datos de entrada, la propia entrada:

<p align="center"><img src="https://www.researchgate.net/publication/311995136/figure/fig2/AS:749324690534400@1555664336772/Auto-encoder-Source-Wikipedia.jpg" height="20%"></p>

Entre otras funcionalidades, se incluyen las siguientes:

- **Reducción de dimensionalidad no lineal**: Una vez entrenado el modelo, si nos quedamos con la activación de la capa oculta (hidden en la figura), tenemos una representación con menor dimensionalidad habiendo realizado una transformación no lineal de los datos.

- **Reducción de ruido**: Una vez entrenado el modelo, si nos quedamos con la activación de la capa de salida (output en la figura), tenemos una reconstrucción de nuestro dato de entrada (input) habiendo filtrado lo más relevante para identificar el dato.

- **Generación de datos sintéticos**: Una vez entrenado el modelo, si generamos números aleatorios para introducirlos como entrada a la capa de decodificación, estaremos generando nuevos datos.

- **Detección de anomalías**: Si un autoencoder se entrena con datos "normales", cuando aparece un dato anómalo o extraño, la reconstrucción del autoencoder será deficiente. Puede ser un indicativo de que algo está fallando con el dato de entrada.

Vamos a explorar cómo funciona un autoencoder en Keras con dos problemas: **MNIST** y **Activos financieros**.

In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt
from time import time
import shutil
import yfinance as yf

# <font color="#CA3532">MNIST</font>

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

print(train_images.shape)
print(train_labels.shape)
print(train_labels)

print(test_images.shape)
print(test_labels.shape)
print(test_labels)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(train_labels[i])

**Preprocesamiento de datos**

In [ ]:
train_images = train_images / 255
test_images = test_images / 255

In [ ]:
plt.imshow(train_images[0], plt.cm.binary, vmin=0, vmax=1)
plt.colorbar()
plt.show()

## <font color="#CA3532">Diseño del modelo</font>

In [ ]:
input_layer = tf.keras.layers.Input(shape=(28,28))
flatten_layer = tf.keras.layers.Flatten()(input_layer)
encoded_layer = tf.keras.layers.Dense(256, activation="relu")(flatten_layer)
decoded_layer = tf.keras.layers.Dense(784, activation="linear")(encoded_layer)

## <font color="#CA3532">Entrenamiento del modelo</font>

In [ ]:
batch_size = 100
learning_rate = 1e-3

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [ ]:
autoencoder = tf.keras.Model(inputs=input_layer, outputs=decoded_layer)
autoencoder.compile(optimizer=optimizer, loss="mean_squared_error")
autoencoder.summary()

In [ ]:
h = autoencoder.fit(train_images, train_images.reshape(-1, 28*28),
                    batch_size=batch_size,
                    validation_data=(test_images, test_images.reshape(-1, 28*28)),
                    epochs=30)

In [ ]:
plt.figure(figsize=(10, 3))
plt.plot(h.history["loss"], '.-', label="Train")
plt.plot(h.history["val_loss"], '.-', label="Test")
plt.legend()
plt.grid()
plt.show()

In [ ]:
decoded_imgs = autoencoder.predict(train_images)

In [ ]:
for i, (original, decoded) in enumerate(zip(train_images, decoded_imgs)):
    plt.subplot(1,2,1)
    plt.imshow(original)
    plt.subplot(1,2,2)
    plt.imshow(decoded.reshape(28,28))
    plt.show()
    if i == 10:
        break

# <font color="#CA3532">Activos</font>

Tenemos una lista de activos. Vamos a procesarlos.

In [ ]:
lista_activos = ['A', 'AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE', 'ADM', 'ADSK',
                 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIZ', 'AKAM', 'ALL', 'ALLE',
                 'AMAT', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN', 'AON', 'APA', 'APD',
                 'APH', 'APTV', 'AVB', 'AVGO', 'AVY', 'AXP', 'AZO', 'BA', 'BAC',
                 'BALL', 'BAX', 'BBWI', 'BBY', 'BDX', 'BEN', 'BK', 'BKNG',
                 'BLK', 'BMY', 'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH',
                 'CAT', 'CB', 'CBRE', 'CCI', 'CCL', 'CF', 'CHRW', 'CI', 'CINF',
                 'CL', 'CLX', 'CMA', 'CMCSA', 'CMG', 'CMI', 'CMS', 'CNP', 'COF',
                 'COP', 'COST', 'CPB', 'CRM', 'CSCO', 'CTAS', 'CTRA', 'CTSH',
                 'CVS', 'CVX', 'D', 'DAL', 'DE', 'DFS', 'DG', 'DGX', 'DHI', 'DHR',
                 'DIS', 'DLTR', 'DOV', 'DRI', 'DTE', 'DUK', 'DVA', 'EA', 'EBAY',
                 'ECL', 'ED', 'EFX', 'EIX', 'EL', 'ELV', 'EMN', 'EMR', 'EOG', 'EQR',
                 'EQT', 'ES', 'ESS', 'ETN', 'ETR', 'EW', 'EXC', 'EXPD', 'EXPE', 'F',
                 'FAST', 'FCX', 'FDX', 'FE', 'FFIV', 'FI', 'FIS', 'FITB', 'FMC',
                 'FSLR', 'GD', 'GE', 'GEN', 'GILD', 'GIS', 'GL', 'GLW', 'GM', 'GOOG',
                 'GOOGL', 'GPC', 'GS', 'GWW', 'HAL', 'HAS', 'HBAN', 'HD', 'HES', 'HIG',
                 'HON', 'HPQ', 'HRL', 'HST', 'HSY', 'HUM', 'IBM', 'ICE', 'IFF', 'INTC',
                 'INTU', 'IP', 'IPG', 'IRM', 'ISRG', 'ITW', 'IVZ', 'JCI', 'JNJ',
                 'JNPR', 'JPM', 'K', 'KDP', 'KEY', 'KIM', 'KLAC', 'KMB', 'KMI', 'KMX',
                 'KO', 'KR', 'L', 'LDOS', 'LEN', 'LH', 'LHX', 'LLY', 'LMT', 'LNC',
                 'LOW', 'LRCX', 'LUV', 'LYB', 'MA', 'MAS', 'MCD', 'MCHP', 'MCK', 'MCO',
                 'MDT', 'MET', 'META', 'MHK', 'MKC', 'MLM', 'MMC', 'MMM', 'MO', 'MOS',
                 'MPC', 'MRK', 'MRO', 'MS', 'MSFT', 'MSI', 'MTB', 'NDAQ', 'NEE', 'NEM',
                 'NFLX', 'NI', 'NKE', 'NOC', 'NRG', 'NSC', 'NTAP', 'NTRS', 'NUE',
                 'NVDA', 'NWL', 'NWSA', 'OKE', 'OMC', 'ORCL', 'ORLY', 'OXY', 'PARA',
                 'PAYX', 'PCAR', 'PCG', 'PEG', 'PFE', 'PG', 'PGR', 'PH', 'PHM',
                 'PLD', 'PM', 'PNC', 'PNR', 'PNW', 'PPG', 'PPL', 'PRU', 'PSA', 'PSX',
                 'PTC', 'PWR', 'PXD', 'QCOM', 'RCL', 'RF', 'RHI', 'RL', 'ROK', 'ROP',
                 'ROST', 'RSG', 'RTX', 'RVTY', 'SBUX', 'SCHW', 'SEE', 'SHW', 'SJM',
                 'SLB', 'SNA', 'SO', 'SPG', 'SPGI', 'SRE', 'STT', 'STX', 'STZ', 'SWK',
                 'SYK', 'SYY', 'T', 'TAP', 'TEL', 'TFC', 'TGT', 'TJX', 'TMO', 'TPR',
                 'TROW', 'TRV', 'TSCO', 'TSN', 'TT', 'TXT', 'UHS', 'UNH', 'UNP', 'UPS',
                 'URI', 'USB', 'V', 'VFC', 'VLO', 'VMC', 'VRSN', 'VTR', 'VZ', 'WAT',
                 'WEC', 'WELL', 'WFC', 'WHR', 'WM', 'WMB', 'WMT', 'WY', 'WYNN', 'XOM',
                 'XRAY', 'XYL', 'YUM', 'ZBH', 'ZION', 'ZTS']

In [ ]:
precios = yf.download(lista_activos, start="2022-01-01", end="2023-01-01")["Adj Close"]
retornos = np.log(precios).diff().dropna()

Vamos a hacer una exploración sencilla de los activos. Para ello, vamos a aplicar PCA sobre la matriz de correlación para intentar representar los activos en 2D.

In [ ]:
matriz_corr = retornos.corr()

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca_2d = pca.fit_transform(matriz_corr)

Con esta transformación, mantenemos un 72.96% de la varianza explicada.

In [ ]:
pca.explained_variance_ratio_.sum()

In [ ]:
plt.plot(pca_2d[:, 0], pca_2d[:, 1], '.')
plt.show()

## <font color="#CA3532">Autoencoder como reducción de dimensionalidad</font>

In [ ]:
input_layer = tf.keras.layers.Input(shape=(matriz_corr.shape[1]))
flatten_layer = tf.keras.layers.Flatten()(input_layer)
encoded_layer = tf.keras.layers.Dense(2, activation="relu", bias_initializer="ones")(flatten_layer)
decoded_layer = tf.keras.layers.Dense(matriz_corr.shape[1], activation="linear")(encoded_layer)

In [ ]:
batch_size = 20
learning_rate = 1e-3

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [ ]:
autoencoder = tf.keras.Model(inputs=input_layer, outputs=decoded_layer)
autoencoder.compile(optimizer=optimizer, loss="mean_squared_error")
autoencoder.summary()

In [ ]:
h = autoencoder.fit(matriz_corr, matriz_corr,
                    batch_size=batch_size,
                    epochs=200)

In [ ]:
plt.figure(figsize=(10, 3))
plt.plot(h.history["loss"], '.-', label="Train")
plt.legend()
plt.grid()
plt.show()

In [ ]:
matriz_corr_decoded = autoencoder.predict(matriz_corr)

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1,2,1)
plt.title("PCA")
plt.plot(pca_2d[:, 0], pca_2d[:, 1], '.', alpha=0.5)
plt.subplot(1,2,2)
plt.title("AUTOENCODER")
plt.plot(matriz_corr_decoded[:, 0], matriz_corr_decoded[:, 1], '.', alpha=0.5)
plt.show()

## <font color="#CA3532">Autoencoder como Denoising</font>

In [ ]:
input_layer = tf.keras.layers.Input(shape=(matriz_corr.shape[1]))
flatten_layer = tf.keras.layers.Flatten()(input_layer)
encoded_layer = tf.keras.layers.Dense(20, activation="relu", bias_initializer="ones")(flatten_layer)
decoded_layer = tf.keras.layers.Dense(matriz_corr.shape[1], activation="linear")(encoded_layer)

In [ ]:
batch_size = 20
learning_rate = 1e-3

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [ ]:
autoencoder = tf.keras.Model(inputs=input_layer, outputs=decoded_layer)
autoencoder.compile(optimizer=optimizer, loss="mean_squared_error")
autoencoder.summary()

In [ ]:
h = autoencoder.fit(matriz_corr, matriz_corr,
                    batch_size=batch_size,
                    epochs=200)

In [ ]:
plt.figure(figsize=(10, 3))
plt.plot(h.history["loss"], '.-', label="Train")
plt.legend()
plt.grid()
plt.show()

In [ ]:
matriz_corr_decoded = autoencoder.predict(matriz_corr)

In [ ]:
pca_decoded = PCA(n_components=2)
pca_2d_decoded = pca_decoded.fit_transform(matriz_corr_decoded)

In [ ]:
pca_decoded.explained_variance_ratio_.sum()

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1,2,1)
plt.title("Matriz Corr\n"+str(pca.explained_variance_ratio_.sum()))
plt.plot(pca_2d[:, 0], pca_2d[:, 1], '.', alpha=0.5)
plt.subplot(1,2,2)
plt.title("Matriz Corr Decoded\n"+str(pca_decoded.explained_variance_ratio_.sum()))
plt.plot(pca_2d_decoded[:, 0], pca_2d_decoded[:, 1], '.', alpha=0.5)
plt.show()

**Preguntas abiertas**

- ¿Qué sucede con la transformación de la matriz de correlación? ¿Qué representa?
- ¿Y si ahora pruebo a hacer HRP o cualquier otra estrategia de clustering con esta nueva matriz de correlación decodificada?
- ¿Los activos que según la matriz de correlación estaban cerca siguen estando cerca ahora?

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
dbscan = DBSCAN(eps=0.6, min_samples=8)
clusters = dbscan.fit_predict(matriz_corr_decoded)

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1,2,1)
plt.title("Matriz Corr\n"+str(pca.explained_variance_ratio_.sum()))
for c in np.unique(clusters):
  if c != -1:
    plt.plot(pca_2d[c==clusters, 0], pca_2d[c==clusters, 1], '.', alpha=0.5)
  else:
    plt.plot(pca_2d[c==clusters, 0], pca_2d[c==clusters, 1], '.', color="gray", alpha=0.1)
plt.subplot(1,2,2)
plt.title("Matriz Corr Decoded\n"+str(pca_decoded.explained_variance_ratio_.sum()))
for c in np.unique(clusters):
  if c != -1:
    plt.plot(pca_2d_decoded[c==clusters, 0], pca_2d_decoded[c==clusters, 1], '.', alpha=0.5)
  else:
    plt.plot(pca_2d_decoded[c==clusters, 0], pca_2d_decoded[c==clusters, 1], '.', color="gray", alpha=0.1)
plt.show()

In [ ]:
dbscan = DBSCAN(eps=1.0, min_samples=8)
clusters = dbscan.fit_predict(matriz_corr)

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1,2,1)
plt.title("Matriz Corr\n"+str(pca.explained_variance_ratio_.sum()))
for c in np.unique(clusters):
  if c != -1:
    plt.plot(pca_2d[c==clusters, 0], pca_2d[c==clusters, 1], '.', alpha=0.5)
  else:
    plt.plot(pca_2d[c==clusters, 0], pca_2d[c==clusters, 1], '.', color="gray", alpha=0.1)
plt.subplot(1,2,2)
plt.title("Matriz Corr Decoded\n"+str(pca_decoded.explained_variance_ratio_.sum()))
for c in np.unique(clusters):
  if c != -1:
    plt.plot(pca_2d_decoded[c==clusters, 0], pca_2d_decoded[c==clusters, 1], '.', alpha=0.5)
  else:
    plt.plot(pca_2d_decoded[c==clusters, 0], pca_2d_decoded[c==clusters, 1], '.', color="gray", alpha=0.1)
plt.show()